In [1]:
import tabula
import pandas

In [2]:
filepath="./Rec_Task/EICHERMOT.pdf"

In [3]:
tables = tabula.read_pdf(filepath, pages="all", stream=True, guess=False, area=[(195.06903282165527, 306.93918800354004, 275.68719795227054, 550.25825340271), (263.7816779327393, 53.94688758850098, 343.3998430633545, 298.0100479888916), (578.5338634490967, 54.690982589721685, 667.8252635955811, 549.5141584014892)])

In [4]:
len(tables)

3

In [5]:
def is_not_nan(val):
    return val == val

def concat_rows(tbl, common_col):
    tmp_df = pandas.DataFrame(columns=tbl.columns)
    ind = -1
    for index, row in tbl.iterrows():
        if(is_not_nan(row[common_col])):
            ind+=1
            tmp_df.loc[ind]=['']*len(tbl.columns)
        for col in tbl.columns:
            tmp_df.loc[ind][col]+=" "+str(row[col] if is_not_nan(row[col]) else "")
            tmp_df.loc[ind][col]=tmp_df.loc[ind][col].strip()
    return tmp_df

def merge_col_row(tbl, last_row):
    col = [str(i if is_not_nan(i) else "") for i in tbl.columns.to_list()]
    for row_ind in range(last_row+1):
        for col_ind in range(len(col)):
            row_item = tbl.loc[row_ind][col_ind]
            col[col_ind] += " " if len(col[col_ind]) > 0 and col[col_ind][-1]!="/" and col[col_ind][-1]!="(" and col[col_ind][-1]!="_" else ""
            col[col_ind] += str(row_item) if is_not_nan(row_item) else ""
            col[col_ind] = col[col_ind].strip()
    
    tbl.columns = col
    tbl = tbl.iloc[last_row+1:].reset_index(drop=True)
    return tbl

def populate_unnamed_col(tbl):
    col = [str(i if is_not_nan(i) and i.find("Unnamed:") == -1 else "") for i in t3.columns.to_list()]
    ind = -1
    for i in range(0,len(col)):
        if col[i] == '':
            ind = i
        else:
            if ind != -1:
                col[i] = col[i]+"_"
                for j in range(ind, i):
                    col[j] = col[i]
                ind = -1
    tbl.columns = col
    return tbl

In [6]:
t1 = tables[0]

In [7]:
t1

,Sl.,Name,Chairman/
0,No.,NaN,Member
1,1.,Mr Siddhartha Lal (Managing Director &,Chairman
2,NaN,Chief Executive Officer),NaN
3,2.,Mr S. Sandilya (Chairman and Non-Executive,Member
4,NaN,Independent Director),NaN
5,3.,Mr Lalit Malik (Chief Financial Officer),Member


In [8]:
# merge first row and column
# l1 = t1.columns[:3].to_list()
# l2 = t1.iloc[0, 0:].to_list()
# t1.columns = [str(i if i==i else "") + str((" " if i[-1]!="/" else "")+j if j==j else "") for i, j in zip(l1, l2)]
# t1 = t1.iloc[1:].reset_index(drop=True)
# t1

t1

,Sl.,Name,Chairman/
0,No.,NaN,Member
1,1.,Mr Siddhartha Lal (Managing Director &,Chairman
2,NaN,Chief Executive Officer),NaN
3,2.,Mr S. Sandilya (Chairman and Non-Executive,Member
4,NaN,Independent Director),NaN
5,3.,Mr Lalit Malik (Chief Financial Officer),Member


In [9]:
# merge first row and column
t1 = merge_col_row(t1, 0)
t1

,Sl. No.,Name,Chairman/Member
0,1.,Mr Siddhartha Lal (Managing Director &,Chairman
1,NaN,Chief Executive Officer),NaN
2,2.,Mr S. Sandilya (Chairman and Non-Executive,Member
3,NaN,Independent Director),NaN
4,3.,Mr Lalit Malik (Chief Financial Officer),Member


In [10]:
# merge rows based on a particular column
t1 = concat_rows(t1, 'Sl. No.')
t1

,Sl. No.,Name,Chairman/Member
0,1.,Mr Siddhartha Lal (Managing Director & Chief E...,Chairman
1,2.,Mr S. Sandilya (Chairman and Non-Executive Ind...,Member
2,3.,Mr Lalit Malik (Chief Financial Officer),Member


In [11]:
t1.to_dict('records')

[{'Sl. No.': '1.',
  'Name': 'Mr Siddhartha Lal (Managing Director & Chief Executive Officer)',
  'Chairman/Member': 'Chairman'},
 {'Sl. No.': '2.',
  'Name': 'Mr S. Sandilya (Chairman and Non-Executive Independent Director)',
  'Chairman/Member': 'Member'},
 {'Sl. No.': '3.',
  'Name': 'Mr Lalit Malik (Chief Financial Officer)',
  'Chairman/Member': 'Member'}]

In [12]:
t2 = tables[1]
t2

,Sl.,Name,Chairman/,No. of,No. of.1
0,No.,NaN,Member,meetings,meetings
1,NaN,NaN,NaN,held,attended
2,1.,Mr S. Sandilya,Chairman,2,2
3,2.,Mr Siddhartha Lal,Member,2,2
4,3.,Mr Prateek Jalan,Member,2,2


In [13]:
# # merge first two rows and column
# l1 = t2.columns[:5].to_list()
# l2 = t2.iloc[0, 0:].to_list()
# l3 = t2.iloc[1, 0:].to_list()
# tmpColumns= [str(i if i==i else "") + str((" " if i[-1]!="/" else "")+j if j==j else "") + str(" "+k if k==k else "") for i, j, k in zip(l1, l2, l3)]
# # tmpColumns[4] = 'No. of meetings attended'
# t2.columns = tmpColumns
# t2.rename(columns={'No. of.1 meetings attended': 'No. of meetings attended'})
# t2 = t2.iloc[2:].reset_index(drop=True)
# t2

In [14]:
# merge first two rows and column
t2 = merge_col_row(t2, 1)
t2

,Sl. No.,Name,Chairman/Member,No. of meetings held,No. of.1 meetings attended
0,1.,Mr S. Sandilya,Chairman,2,2
1,2.,Mr Siddhartha Lal,Member,2,2
2,3.,Mr Prateek Jalan,Member,2,2


In [15]:
t2.to_dict('records')

[{'Sl. No.': '1.',
  'Name': 'Mr S. Sandilya',
  'Chairman/Member': 'Chairman',
  'No. of meetings held': '2',
  'No. of.1 meetings attended': '2'},
 {'Sl. No.': '2.',
  'Name': 'Mr Siddhartha Lal',
  'Chairman/Member': 'Member',
  'No. of meetings held': '2',
  'No. of.1 meetings attended': '2'},
 {'Sl. No.': '3.',
  'Name': 'Mr Prateek Jalan',
  'Chairman/Member': 'Member',
  'No. of meetings held': '2',
  'No. of.1 meetings attended': '2'}]

In [16]:
t3 = tables[2]
t3

,Name of the Director,Salary,Commission,Perquisites,Unnamed: 0,Service Contract
0,NaN,NaN,NaN,NaN,Tenure,Notice period
1,Mr Siddhartha Lal,3.29,4.2,1.71,5 years (Members at their,3 months' notice or salary in
2,Managing Director,NaN,NaN,NaN,"AGM held on June 18, 2016,",lieu of notice for 3 months or for
3,NaN,NaN,NaN,NaN,approved re-appointment of,such period which falls short of
4,NaN,NaN,NaN,NaN,Mr Siddhartha Lal as Managing,3 months
5,NaN,NaN,NaN,NaN,"Director w.e.f. May 1, 2016 up to",NaN
6,NaN,NaN,NaN,NaN,"April 30, 2021)",NaN


In [17]:
# # merge first row and column
# l1 = t3.columns[:6].to_list()
# l2 = t3.iloc[0, 0:].to_list()
# t3.columns = [(str(i if i==i and i.find("Unnamed:") == -1 else "") + str(" "+j if j==j else "")).strip() for i, j in zip(l1, l2)]
# t3 = t3.iloc[1:].reset_index(drop=True)
# t3

In [18]:
# l1 = t3.columns[:6].to_list()
# col = [str(i if is_not_nan(i) and i.find("Unnamed:") == -1 else "") for i in t3.columns.to_list()]

# # for subheading
# ind = -1
# for i in range(0,len(col)):
#     if col[i] == '':
#         ind = i
#     else:
#         if ind != -1:
#             col[i] = col[i]+"_"
#             for j in range(ind, i):
#                 col[j] = col[i]
#             ind - -1
            
        
# col

t3 = populate_unnamed_col(t3)
t3

,Name of the Director,Salary,Commission,Perquisites,Service Contract_,Service Contract_
0,NaN,NaN,NaN,NaN,Tenure,Notice period
1,Mr Siddhartha Lal,3.29,4.2,1.71,5 years (Members at their,3 months' notice or salary in
2,Managing Director,NaN,NaN,NaN,"AGM held on June 18, 2016,",lieu of notice for 3 months or for
3,NaN,NaN,NaN,NaN,approved re-appointment of,such period which falls short of
4,NaN,NaN,NaN,NaN,Mr Siddhartha Lal as Managing,3 months
5,NaN,NaN,NaN,NaN,"Director w.e.f. May 1, 2016 up to",NaN
6,NaN,NaN,NaN,NaN,"April 30, 2021)",NaN


In [19]:
# a = pandas.MultiIndex.from_product([['Service Contract'],
#                                      ['Tenure', 'Notice Period']])

In [20]:
# col[:-2].append(a)

# col = col[:-2]

# col.extend(a)
# list(a)

In [21]:
# print(col)
# t3.columns = col

In [22]:
# merge first row and column
t3 = merge_col_row(t3, 0)

# t3 = t3.iloc[1:].reset_index(drop=True)
t3

,Name of the Director,Salary,Commission,Perquisites,Service Contract_Tenure,Service Contract_Notice period
0,Mr Siddhartha Lal,3.29,4.2,1.71,5 years (Members at their,3 months' notice or salary in
1,Managing Director,NaN,NaN,NaN,"AGM held on June 18, 2016,",lieu of notice for 3 months or for
2,NaN,NaN,NaN,NaN,approved re-appointment of,such period which falls short of
3,NaN,NaN,NaN,NaN,Mr Siddhartha Lal as Managing,3 months
4,NaN,NaN,NaN,NaN,"Director w.e.f. May 1, 2016 up to",NaN
5,NaN,NaN,NaN,NaN,"April 30, 2021)",NaN


In [23]:
t3 = concat_rows(t3, 'Salary')
t3

,Name of the Director,Salary,Commission,Perquisites,Service Contract_Tenure,Service Contract_Notice period
0,Mr Siddhartha Lal Managing Director,3.29,4.2,1.71,"5 years (Members at their AGM held on June 18,...",3 months' notice or salary in lieu of notice f...


In [24]:
t3.to_dict('records')

[{'Name of the Director': 'Mr Siddhartha Lal Managing Director',
  'Salary': '3.29',
  'Commission': '4.2',
  'Perquisites': '1.71',
  'Service Contract_Tenure': '5 years (Members at their AGM held on June 18, 2016, approved re-appointment of Mr Siddhartha Lal as Managing Director w.e.f. May 1, 2016 up to April 30, 2021)',
  'Service Contract_Notice period': "3 months' notice or salary in lieu of notice for 3 months or for such period which falls short of 3 months"}]

NameError: name 'db' is not defined